In [ ]:
import torch
import torch.optim as optim
from torch.distributions.multivariate_normal import MultivariateNormal

# Custom imports
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
sys.path.insert(0, os.path.dirname(currentdir))
from simulators import get_simulator
from ml.real_nvps import RealNVP, RealNVPs
from ml.umnn_maf_flow import UmnnMafFlow
from ml.ml_helper import fit_conditional_normalizing_flow
from metrics.metrics import compute_roc_auc
from inference.neb import McBiasedEstimator, McUnbiasedEstimator
from utils.noise_distribution import GaussianDistribution

In [ ]:
device = 'cuda'

# Define your simulator

In [ ]:
simulator_name = 'SLCP' # Choose a simulator in ['SLCP', 'IK', '2dMoon']
simulator = get_simulator(simulator_name)

# Train a surrogate model of the simulator 
#### (needed when no likelihood function is known in closed-form)

In [ ]:
# Generate the training dataset
dataset_size = 15000
proposal_distribution = MultivariateNormal(torch.zeros(dataset_size, simulator.xdim), torch.eye(simulator.xdim))
x_train = proposal_distribution.sample()
y_train = simulator.corrupt(x_train)

# Train the surrogate model
flow_length = 4
hidden_layer_dim = 50

surrogate = RealNVPs(flow_length, simulator.ydim, simulator.xdim, hidden_layer_dim).to(device)
optimizer = optim.Adam(surrogate.parameters(), weight_decay=5*10**(-5), lr=1e-4)

training_loss, validation_loss = fit_conditional_normalizing_flow(surrogate, 
                                                                  optimizer, 
                                                                  y_train.to(device), 
                                                                  x_train.to(device), 
                                                                  validation_size=0.1, 
                                                                  early_stopping_patience=10,
                                                                  batch_size=128, 
                                                                  nb_epochs=300)
surrogate.eval();

#### Assess the quality of the surrogate model (the closer to 0.5, the better)

In [ ]:
source_data = proposal_distribution.sample()
true_y = simulator.corrupt(source_data)

noise = GaussianDistribution.sample(dataset_size, dim=simulator.ydim)
regenerated_y = surrogate.invert(noise.to(device), context=source_data.to(device))

print('ROC AUC:', compute_roc_auc(true_y.data.numpy(), regenerated_y.data.cpu().numpy()))

# Source Distribution Estimation 

In [ ]:
# Unseen source distribution
source_distribution = simulator.sample_prior(10000)
# Observations
observations = simulator.corrupt(source_distribution)

#### Definition of the source model
- If the downstream analysis does not require explicit density evaluation under the source model, any generative model can be used and should be preferred over normalizing flows.
    - Note however that normalizing flows may act as useful regularizers for continuous and smooth source distributions.

In [ ]:
density_evaluation_is_required = True

if density_evaluation_is_required:
    """
    This is a heavy model, lighter models are recommended when inference 
    is done in higher dimensions or on more data.
    """
    source_model = UmnnMafFlow(nb_flow=6, 
                               nb_in=simulator.xdim, 
                               cond_in=0,
                               hidden_derivative=[75, 75, 75], 
                               hidden_embedding=[75, 75, 75],
                               embedding_s=10, 
                               nb_steps=20,
                               device=device)
else:
    hidden_layer_dim = 100
    input_noise_dim = simulator.xdim
    source_model = nn.Sequential(
                        nn.Linear(input_noise_dim, hidden_layer_dim),
                        nn.ReLU(),
                        nn.Linear(hidden_layer_dim, hidden_layer_dim),
                        nn.BatchNorm1d(hidden_layer_dim),
                        nn.ReLU(),
                        nn.Linear(hidden_layer_dim, simulator.xdim)
                        ).to(device)
    
optimizer = optim.Adam(source_model.parameters(), weight_decay=0.0, lr=1e-4)

#### Inference

In [ ]:
log_likelihood_fct = lambda y, x : surrogate.compute_ll(y, context=x)[0]

estimator = McBiasedEstimator() # Choose an estimator in ['McBiasedEstimator', 'McUnbiasedEstimator']

nb_epochs = 100
nb_mc_integration_steps = 1024
batch_size = 128
early_stopping=True, # We do not recommend to do early_stopping with
                     # the unbiased estimator which is quite noisy

training_loss, validation_loss = estimator.infer(observations.to(device), 
                                                 source_model, 
                                                 optimizer, 
                                                 log_likelihood_fct, 
                                                 simulator.xdim,
                                                 nb_epochs=nb_epochs, 
                                                 batch_size=batch_size, 
                                                 early_stopping=early_stopping,
                                                 nb_mc_integration_steps=nb_mc_integration_steps)

#### Assess the quality of the learned source distribution (the closer to 0.5, the better)

In [ ]:
noise = GaussianDistribution.sample(10000, dim=simulator.xdim)

# Assess the quality of the source model
regenerated_source_data = source_model(noise.to(device)).data.cpu()
regenerated_observations = simulator.corrupt(regenerated_source_data).data

print('ROC AUC x-space:', compute_roc_auc(regenerated_source_data.numpy(), source_distribution.data.numpy()))
print('ROC AUC y-space:', compute_roc_auc(regenerated_observations.numpy(), observations.data.numpy()))